#Working Code

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.2 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 39.6 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import os

# Set up API key
os.environ["OPENAI_API_KEY"] = "openai_key"

# Agents Definitions
class PDFReaderAgent:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path

    def read_pdf(self):
        try:
            if not os.path.exists(self.pdf_path):
                print(f"File {self.pdf_path} does not exist. Please check the file path.")
                return ""

            # Reading the PDF file
            pdfreader = PdfReader(self.pdf_path)
            raw_text = ''
            for page in pdfreader.pages:
                content = page.extract_text()
                if content:
                    raw_text += content
            return raw_text
        except Exception as e:
            print(f"An error occurred while reading the PDF: {e}")
            return ""

class TextSplitterAgent:
    def __init__(self, chunk_size, chunk_overlap):
        self.text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )

    def split_text(self, raw_text):
        return self.text_splitter.split_text(raw_text)

class EmbeddingAgent:
    def __init__(self):
        self.openai_embeddings = OpenAIEmbeddings()
        self.sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

    def get_openai_embedding(self, texts):
        return self.openai_embeddings

    def get_sentence_transformer_embedding(self, text):
        return self.sentence_transformer.encode(text)

class DocumentSearchAgent:
    def __init__(self, texts, embeddings,embedding_agent):
        self.faiss_index = FAISS.from_texts(texts, embeddings)
        self.embedding_agent = embedding_agent

    def similarity_search(self, query_embedding):

        docs_with_scores = []
        for doc in self.faiss_index.similarity_search(query):
            doc_embedding = self.embedding_agent.get_sentence_transformer_embedding(doc.page_content)
            score = cosine_similarity([query_embedding], [doc_embedding])[0][0]
            docs_with_scores.append((doc, score))
        return docs_with_scores

class FAQMatcherAgent:
    def __init__(self, faq_dict):
        self.faq_dict = faq_dict

    def match_faq(self, query):
        if query in self.faq_dict:
            return self.faq_dict[query], "Exact Match"
        return None, None

class AnswerGeneratorAgent:
    def __init__(self, llm, chain_type):
        self.chain = load_qa_chain(llm, chain_type=chain_type)

    def generate_answer(self, docs, question):
        return self.chain.run(input_documents=docs, question=question)

class SimilarityAgent:
    def calculate_similarity(self, query_embedding, doc_embedding):
        return cosine_similarity([query_embedding], [doc_embedding])[0][0]

# Orchestrator
class Orchestrator:
    def __init__(self, pdf_path, faq_dict, threshold=0.45):
        self.pdf_reader_agent = PDFReaderAgent(pdf_path)
        self.faq_matcher_agent = FAQMatcherAgent(faq_dict)
        self.embedding_agent = EmbeddingAgent()
        self.similarity_agent = SimilarityAgent()
        self.answer_generator_agent = AnswerGeneratorAgent(OpenAI(), "stuff")
        self.threshold = threshold

    def get_response(self, query):
        # Match against FAQ
        faq_response, source = self.faq_matcher_agent.match_faq(query)
        if faq_response:
            return faq_response, source

        # Get embeddings for query and search documents
        query_embedding = self.embedding_agent.get_sentence_transformer_embedding(query)
        raw_text = self.pdf_reader_agent.read_pdf()

        # Text splitting
        text_splitter_agent = TextSplitterAgent(800, 200)
        texts = text_splitter_agent.split_text(raw_text)

        # Document search with similarity calculation
        document_search_agent = DocumentSearchAgent(texts, self.embedding_agent.get_openai_embedding(texts), self.embedding_agent)
        docs_with_scores = document_search_agent.similarity_search(query_embedding)

        # Filter documents that meet the similarity threshold
        docs = [doc for doc, score in docs_with_scores if score >= self.threshold]

        # Check if there are similar documents
        if not docs:
            return "The question is not in the database.", "No Vector Database"

        # Generate response from documents
        response = self.answer_generator_agent.generate_answer(docs, query)
        return response, "Vector Database"

# FAQ data
faq_dict = {
    "What is AWS, and how does it work?": "AWS (Amazon Web Services) provides cloud computing services where you can rent computing power, storage, and networking resources on a pay-as-you-go basis. It allows users to run virtual servers, store files, and deploy applications on a large-scale, global infrastructure.",

    "What services does AWS offer for file storage?": "AWS offers Amazon Simple Storage Service (S3) for storing digital files. It provides scalable, secure, and reliable file storage, allowing users to store and retrieve any amount of data at any time.",

    "Can I host a website on AWS?": "Yes, AWS offers various services to host both static and dynamic websites. You can use Amazon S3 for hosting static websites and Amazon EC2 for dynamic websites with server-side processing.",

    "What databases are supported by AWS?": "AWS supports both relational and NoSQL databases. Amazon RDS is used for relational databases like MySQL, Oracle, and SQL Server, while Amazon DynamoDB is used for NoSQL databases, offering high scalability and performance.",

    "What is the AWS Free Usage Tier, and who can use it?": "The AWS Free Usage Tier allows new AWS users to explore AWS services without incurring any costs for a year. It offers limited free usage for services such as EC2, S3, and RDS, which are ideal for small applications and testing environments.",

    "How can I monitor my AWS resource usage and costs?": "You can monitor your AWS usage and costs via the AWS Billing and Cost Management console. Additionally, AWS CloudWatch helps you track resource usage and set up billing alerts to notify you if your usage exceeds the free tier limits.",

    "How do I create a virtual server on AWS?": "To create a virtual server, go to the Amazon EC2 console, select an Amazon Machine Image (AMI), choose the t1.micro instance type to stay within the free tier, and launch the instance. AWS provisions the virtual server for you.",

    "What is Amazon Elastic Beanstalk, and how is it used?": "Amazon Elastic Beanstalk is an application management service that simplifies the process of deploying, managing, and scaling web applications and services. It supports various programming languages and frameworks like Docker, Java, Python, and more.",

    "How can I create a billing alert to avoid unexpected charges on AWS?": "You can create a billing alert by going to the AWS Billing console and enabling billing alerts under Preferences. Then, use Amazon CloudWatch to set up alarms that will notify you if your estimated charges exceed your desired threshold.",

    "What are the benefits of using Auto Scaling on AWS?": "Auto Scaling automatically adjusts the number of virtual servers based on traffic demands, ensuring that your application has the right amount of resources. It can scale up during peak demand and scale down when demand decreases, optimizing cost-efficiency."
}


# Testing Orchestrator and PDF reading functionality
pdf_path = 'awsgsg-intro.pdf'  # Specify your PDF file path here
orchestrator = Orchestrator(pdf_path=pdf_path, faq_dict=faq_dict)

# Test response generation
query = "How do I make a file public in Amazon S3?"
response, source = orchestrator.get_response(query)

# Output the response for the query
print(f"Response: {response}")
print(f"Source: {source}")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
<ipython-input-6-b2ad8316d4e9>:51: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  self.openai_embeddings = OpenAIEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-6-b2ad8316d4e9>:101: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  self.answer_generator_agent = AnswerGeneratorAgent(OpenAI(), "stuff")
<ipython-input-6-b2ad8316d4e9>:85: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and questi

Response:  To make a file public in Amazon S3, you can follow these steps:

1. Open the Amazon S3 console at https://console.aws.amazon.com/s3/.
2. Select the bucket that contains the file you want to make public.
3. Right-click on the file and select "Make Public."
4. Confirm the action by clicking "OK."
5. Your file can now be accessed by anyone on the internet using its URL.
Source: Vector Database


In [ ]:
import gradio as gr

# Gradio interface to interact with Orchestrator
def gradio_interface(query):
    response, source = orchestrator.get_response(query)
    return response, source

# Creating Gradio Interface
interface = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs=["text", "text"],
    title="DocuSearch System",
    description="Ask questions related to product documentation and get instant, concise answers. The system uses NLP and advanced search to retrieve and summarize relevant sections from the documentation."
)

# Launch the Gradio web app
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80100dcc2c91fe35ac.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


##Exact Match:

1.   Can I host a website on AWS?
2.   What databases are supported by AWS?
3.   What are the benefits of using Auto Scaling on AWS?

## Vector Database

1.   How do I make a file public in Amazon S3?
2.   What is the free usage tier for Amazon EC2?
3.   How do I create a billing alert for AWS usage?

## Not in Vector Database

1. How do I set up an AWS Lambda function?
2. What are the differences between Amazon Aurora and Amazon RDS?
3. What is the process for configuring a custom VPC peering connection?



